In [204]:
import matplotlib.pyplot as plt
import numpy  as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict,cross_validate, GridSearchCV

In [219]:
housing = pd.read_csv('../data/Ames_Housing_Price_Data.csv', index_col = 0)
real_estate = pd.read_csv('../data/Ames_Real_Estate_Data.csv')
housing_df = pd.merge(housing,real_estate,left_on='PID',right_on='MapRefNo',how='inner')

#### Merging two dataset has 21 observations in difference (left/inner), so using 2558 observations to analyze housing price

In [214]:
real_estate.columns

Index(['MapRefNo', 'GeoRefNo', 'Tier', 'Range', 'Prop_Addr', 'ZngCdPr',
       'ZngCdSc', 'ZngOLPr', 'ZngOLSc', 'ClassPr_S', 'ClassSc_S', 'Legal_Pr',
       'SchD_S', 'TxD_S', 'MA_Ownr1', 'MA_Ownr2', 'MA_Line1', 'MA_Line2',
       'MA_City', 'MA_State', 'MA_Zip1', 'MA_Zip2', 'Rcrd_Yr', 'Rcrd_Mo',
       'Inst1_No', 'Inst1_Yr', 'Inst1_Mo', 'Inst1TPr', 'LndAc_S', 'ImpAc_S',
       'OthAc_S', 'TtlVal_AsrYr', 'ValType', 'X1TPr_D', 'X1TSc_D', 'X2TPr_D',
       'X2TSc_D', 'X1TPr_S', 'X1TSc_S', 'X2TPr_S', 'X2TSc_S', 'LndAcX1S',
       'ImpAcX1S', 'ImpAcX2S', 'HSTtl_D', 'MilVal_D', 'HSTtl_S', 'MilVal_S',
       'AcreX_S1', 'AcreGr', 'AcreNt_S', 'Neighborhood', 'LotArea', 'ParType',
       'BldgNo_S', 'DwlgNo_S', 'BldgType', 'YrBuilt', 'HouseStyle',
       'Foundation', 'RoofMatl', 'Ext1', 'Ext2', 'MasVnrType', 'Heating',
       'Central Air', 'GLA', 'TtlBsmtSF', 'TotRmsAbvGrd', 'Fireplaces',
       'PoolArea', 'GarageType', 'GarYrBlt', 'Cars', 'GarageArea',
       'YrSold_YYYY', 'MoSold_MM', '

In [190]:
pd.value_counts(housing_df['MapRefNo'] == housing_df['GeoRefNo']) # MapRefNo same as GeoRefNo, so drop these two columns

True    2603
dtype: int64

In [194]:
housing_df.shape

(2603, 172)

## SalePrice difference??

In [199]:
housing_df['SalePrice_y']

0       155000.0
1       174400.0
2       170000.0
3            NaN
4       139000.0
          ...   
2598         NaN
2599         NaN
2600         NaN
2601         NaN
2602         NaN
Name: SalePrice_y, Length: 2603, dtype: float64

In [200]:
housing_df['SalePrice_x']

0       126000
1       139500
2       139500
3       124900
4       114000
         ...  
2598    121000
2599    139600
2600    145000
2601    217500
2602    215000
Name: SalePrice_x, Length: 2603, dtype: int64

In [195]:
housing_df[housing_df['PID'].duplicated()]

,PID,GrLivArea,SalePrice_x,MSSubClass,MSZoning,LotFrontage,LotArea_x,Street,Alley,LotShape,...,PA-Nmbr,PA-PreD,PA-Strt,PA-StSfx,PA-PostD,PA-UnTyp,PA-UntNo,Date,Source,NmbrBRs
2,905476230,1049,139500,120,RL,42.0,4235,Pave,NaN,Reg,...,3416,NaN,WEST,ST,NaN,NaN,NaN,10-Jul-20,Ames City Assessor,2.0
39,902206020,492,61500,30,RM,52.0,7830,Pave,NaN,Reg,...,118,E,11TH,ST,NaN,NaN,NaN,10-Jul-20,Ames City Assessor,1.0
42,527455250,988,129250,20,RL,65.0,7150,Pave,NaN,Reg,...,1115,NaN,28TH,ST,NaN,NaN,NaN,10-Jul-20,Ames City Assessor,3.0
45,914452190,1080,134000,20,RL,71.0,9187,Pave,NaN,Reg,...,302,NaN,OPAL,CIR,NaN,NaN,NaN,10-Jul-20,Ames City Assessor,3.0
48,532376250,925,133500,20,RL,NaN,8638,Pave,NaN,IR1,...,4015,NaN,TORONTO,ST,NaN,NaN,NaN,10-Jul-20,Ames City Assessor,2.0
299,903429110,1944,179900,70,RM,68.0,9928,Pave,Grvl,Reg,...,716,NaN,HODGE,AVE,NaN,NaN,NaN,10-Jul-20,Ames City Assessor,3.0
411,527214060,2698,535000,60,RL,82.0,16052,Pave,NaN,IR1,...,4008,NaN,FRANCES E DAVIS,CT,NaN,NaN,NaN,10-Jul-20,Ames City Assessor,4.0
691,903225040,1274,135000,50,RM,50.0,6125,Pave,NaN,Reg,...,1217,NaN,CURTISS,AVE,NaN,NaN,NaN,10-Jul-20,Ames City Assessor,3.0
699,923226300,1092,85500,160,RM,21.0,1504,Pave,NaN,Reg,...,609,NaN,MEADOW,PL,NaN,NaN,NaN,10-Jul-20,Ames City Assessor,3.0
700,923226300,1092,85500,160,RM,21.0,1504,Pave,NaN,Reg,...,609,NaN,MEADOW,PL,NaN,NaN,NaN,10-Jul-20,Ames City Assessor,3.0


In [203]:
housing_df[housing_df['PID']==907135040][['SalePrice_x','SalePrice_y','SaleCon']]

,SalePrice_x,SalePrice_y
2513,145000,164250.0
2514,145000,158855.0


In [189]:
print(len(set(housing['PID']))) # 2579 unique observations
print(len(set(housing_df['PID']))) # 2558 unique observations
# housing_df.drop(['Map'])

2579
2558


In [173]:
housing_df[housing_df['PID']==909276070]

,PID,GrLivArea,SalePrice_x,MSSubClass,MSZoning,LotFrontage,LotArea_x,Street,Alley,LotShape,...,PA-Nmbr,PA-PreD,PA-Strt,PA-StSfx,PA-PostD,PA-UnTyp,PA-UntNo,Date,Source,NmbrBRs
1834,909276070,1717,194000,50,RL,80.0,12400,Pave,NaN,Reg,...,2022,NaN,GREELEY,ST,NaN,NaN,NaN,10-Jul-20,Ames City Assessor,2.0
1835,909276070,1717,194000,50,RL,80.0,12400,Pave,NaN,Reg,...,2022,NaN,GREELEY,ST,NaN,NaN,NaN,10-Jul-20,Ames City Assessor,2.0


In [174]:
len(set(housing_df['PID']))

2559

In [136]:
housing[housing['PID']==909276070]

,PID,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
817,909276070,1717,194000,50,RL,80.0,12400,Pave,NaN,Reg,...,113,0,NaN,NaN,NaN,0,2,2006,WD,Normal
1,909276070,1717,194000,50,RL,80.0,12400,Pave,NaN,Reg,...,113,0,NaN,NaN,NaN,0,2,2006,WD,Normal


In [4]:
correlation = housing.corr()['SalePrice']
# corr_matrix['SalePrice']
correlation.sort_values()

PID             -0.226276
EnclosedPorch   -0.123235
KitchenAbvGr    -0.113977
OverallCond     -0.101091
MSSubClass      -0.084587
BsmtHalfBath    -0.033802
LowQualFinSF    -0.030835
MiscVal         -0.018877
YrSold          -0.001376
MoSold           0.012591
BsmtFinSF2       0.022520
PoolArea         0.030844
3SsnPorch        0.031260
ScreenPorch      0.121463
BedroomAbvGr     0.151806
BsmtUnfSF        0.167519
2ndFlrSF         0.263144
LotArea          0.274443
HalfBath         0.285875
BsmtFullBath     0.291080
OpenPorchSF      0.306540
WoodDeckSF       0.333232
LotFrontage      0.358577
BsmtFinSF1       0.460792
Fireplaces       0.488184
TotRmsAbvGrd     0.490219
MasVnrArea       0.500964
YearRemodAdd     0.514309
GarageYrBlt      0.520763
FullBath         0.534982
YearBuilt        0.544370
GarageArea       0.634992
GarageCars       0.638801
1stFlrSF         0.642613
TotalBsmtSF      0.652222
GrLivArea        0.719981
OverallQual      0.790501
SalePrice        1.000000
Name: SalePr

Danny

In [75]:
housing3 = pd.concat([housing[housing.columns[53:]],housing['SalePrice']],axis=1)
housing3.columns

Index(['KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional',
       'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'PoolQC', 'Fence', 'MiscFeature', 'MiscVal',
       'MoSold', 'YrSold', 'SaleType', 'SaleCondition', 'SalePrice'],
      dtype='object')

In [6]:
housing_features = housing[housing.columns[53:]]
housing_features = housing_features.rename(columns= {"3SsnPorch":"X3SsnPorch"})
housing_featuresClass = housing_features[['KitchenQual','Functional','FireplaceQu','GarageType','GarageFinish','GarageQual',\
                                         'GarageCond','PavedDrive','PoolQC','Fence','MiscFeature','SaleType','SaleCondition']]
housing_featuresQuan = housing_features[['KitchenAbvGr','TotRmsAbvGrd','Fireplaces','GarageYrBlt','GarageCars','GarageArea',\
                                        'WoodDeckSF','OpenPorchSF','EnclosedPorch','X3SsnPorch','ScreenPorch','PoolArea',\
                                         'MiscVal','MoSold','YrSold']]

In [59]:
housing_features.columns

Index(['KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional',
       'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch',
       'X3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC', 'Fence',
       'MiscFeature', 'MiscVal', 'MoSold', 'YrSold', 'SaleType',
       'SaleCondition'],
      dtype='object')

In [7]:
for feature1 in housing_featuresClass.columns:
    print(housing3[feature1].value_counts(), housing_featuresClass[feature1].isnull().value_counts())
    print('*'*30)

TA    1358
Gd    1017
Ex     142
Fa      62
Po       1
Name: KitchenQual, dtype: int64 False    2580
Name: KitchenQual, dtype: int64
******************************
Typ     2399
Min2      66
Min1      57
Mod       32
Maj1      18
Maj2       7
Sal        1
Name: Functional, dtype: int64 False    2580
Name: Functional, dtype: int64
******************************
Gd    619
TA    572
Fa     70
Po     43
Ex     35
Name: FireplaceQu, dtype: int64 False    1339
True     1241
Name: FireplaceQu, dtype: int64
******************************
Attchd     1527
Detchd      716
BuiltIn     153
Basment      27
2Types       21
CarPort       9
Name: GarageType, dtype: int64 False    2453
True      127
Name: GarageType, dtype: int64
******************************
Unf    1112
RFn     718
Fin     621
Name: GarageFinish, dtype: int64 False    2451
True      129
Name: GarageFinish, dtype: int64
******************************
TA    2312
Fa     110
Gd      22
Po       4
Ex       3
Name: GarageQual, dtype: int64 F

In [8]:
for feature2 in housing_featuresQuan.columns:
    print(housing_featuresQuan[feature2].isnull().value_counts())
    print('*'*30)

False    2580
Name: KitchenAbvGr, dtype: int64
******************************
False    2580
Name: TotRmsAbvGrd, dtype: int64
******************************
False    2580
Name: Fireplaces, dtype: int64
******************************
False    2451
True      129
Name: GarageYrBlt, dtype: int64
******************************
False    2579
True        1
Name: GarageCars, dtype: int64
******************************
False    2579
True        1
Name: GarageArea, dtype: int64
******************************
False    2580
Name: WoodDeckSF, dtype: int64
******************************
False    2580
Name: OpenPorchSF, dtype: int64
******************************
False    2580
Name: EnclosedPorch, dtype: int64
******************************
False    2580
Name: X3SsnPorch, dtype: int64
******************************
False    2580
Name: ScreenPorch, dtype: int64
******************************
False    2580
Name: PoolArea, dtype: int64
******************************
False    2580
Name: MiscVal, dtype: in

In [13]:
housing3['Fireplaces'].value_counts()

0    1241
1    1132
2     195
3      11
4       1
Name: Fireplaces, dtype: int64

In [19]:
housing3['FireplaceQu'] = housing3['FireplaceQu'].fillna('None')

In [84]:
# Impute None to Nan value in columns
impute_feature = housing3[['FireplaceQu','GarageType','GarageYrBlt','GarageFinish','GarageQual','GarageCond']]
for feature in impute_feature.columns:
    housing3[feature] = housing3[feature].fillna('None')

In [33]:
# housing3['GarageType'].value_counts()

Attchd     1527
Detchd      716
BuiltIn     153
None        127
Basment      27
2Types       21
CarPort       9
Name: GarageType, dtype: int64

In [85]:
housing3[housing3['GarageType'].isnull() == True].T

""
KitchenAbvGr
KitchenQual
TotRmsAbvGrd
Functional
Fireplaces
FireplaceQu
GarageType
GarageYrBlt
GarageFinish
GarageCars


In [88]:
# Drop the Nan from GarageCars column (1 observation)
housing3.drop(housing3[housing3['GarageCars'].isnull() == True].index,inplace=True)
housing3.columns

Index(['KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional',
       'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'PoolQC', 'Fence', 'MiscFeature', 'MiscVal',
       'MoSold', 'YrSold', 'SaleType', 'SaleCondition', 'SalePrice'],
      dtype='object')

In [87]:
housing3.isnull().sum()

KitchenAbvGr        0
KitchenQual         0
TotRmsAbvGrd        0
Functional          0
Fireplaces          0
FireplaceQu         0
GarageType          0
GarageYrBlt         0
GarageFinish        0
GarageCars          0
GarageArea          0
GarageQual          0
GarageCond          0
PavedDrive          0
WoodDeckSF          0
OpenPorchSF         0
EnclosedPorch       0
3SsnPorch           0
ScreenPorch         0
PoolArea            0
PoolQC           2568
Fence            2053
MiscFeature      2480
MiscVal             0
MoSold              0
YrSold              0
SaleType            0
SaleCondition       0
SalePrice           0
dtype: int64

In [96]:
X.isnull().sum().sum()

0

#### PoolArea, PoolQC, Fence, MiscFeature, MiscVal will be drop due to too many missing value

In [89]:
housing_features2 = housing3.drop(['PoolArea','PoolQC','Fence','MiscFeature','MiscVal','SalePrice'],axis=1)
len(housing_features2.columns)

23

In [99]:
X = pd.get_dummies(housing_features2, drop_first=False, dummy_na=True)
price = housing3['SalePrice']
price.shape

(2577,)

In [80]:
X.reset_index()

,index,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,...,SaleType_VWD,SaleType_WD,SaleType_nan,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SaleCondition_nan
0,1,1,4,1,1939.0,2.0,399.0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
1,2,1,5,0,1984.0,1.0,266.0,0,105,0,...,0,1,0,0,0,0,0,1,0,0
2,3,1,5,0,1930.0,1.0,216.0,154,0,42,...,0,1,0,0,0,0,0,1,0,0
3,4,1,6,0,1940.0,1.0,281.0,0,0,168,...,0,1,0,0,0,0,0,1,0,0
4,5,1,6,0,2001.0,2.0,528.0,0,45,0,...,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2575,759,1,4,1,1916.0,1.0,192.0,0,98,0,...,0,1,0,0,0,0,0,1,0,0
2576,760,1,8,1,1955.0,2.0,452.0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
2577,761,2,8,0,1949.0,3.0,871.0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
2578,762,1,7,1,2000.0,2.0,486.0,193,96,0,...,0,1,0,0,0,0,0,1,0,0


In [103]:
lin_reg = LinearRegression().fit(X,price)
lin_reg.score(X,price)

0.7938900509000838

In [206]:
tree_reg = DecisionTreeRegressor(max_depth=10).fit(X,price)
tree_reg.score(X,price)

0.9163681782837603

In [211]:
print(tree_reg.feature_importances_)

[2.44369127e-04 2.94200942e-02 7.29446726e-02 5.24454195e-01
 4.11080155e-02 3.24870852e-02 2.07554070e-02 2.96993546e-03
 2.70031274e-04 2.25643379e-03 1.68328308e-02 1.71012705e-03
 7.83329042e-02 8.15279263e-04 3.58519497e-03 0.00000000e+00
 9.26062418e-02 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.35320535e-06 5.52558851e-06 3.55596530e-04 3.40907703e-04
 1.64920718e-04 0.00000000e+00 3.20564018e-05 0.00000000e+00
 1.25090663e-03 2.20089084e-03 0.00000000e+00 2.11497931e-03
 0.00000000e+00 8.20969638e-05 2.49831468e-03 0.00000000e+00
 8.12768963e-05 5.45616612e-04 1.15481061e-02 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 2.83640317e-04
 0.00000000e+00 3.39241063e-06 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.73505341e-03
 0.00000000e+00 3.05708376e-04 0.00000000e+00 4.63561924e-04
 0.00000000e+00 0.000000

In [210]:
forest_reg = RandomForestRegressor(n_estimators=100,max_features=5).fit(X,price)
forest_reg.score(X,price)

0.9675098297874761